# Calculate sigmav vs. mDM for simulated DM datasets

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

import numpy as np
import astropy.units as u
from astropy.coordinates import SkyCoord, Angle

from gammapy.maps import MapAxis, RegionNDMap
from gammapy.modeling.models import PointSpatialModel
from gammapy.modeling.models import SkyModel, Models
from gammapy.irf import load_cta_irfs
from gammapy.modeling.models import EBLAbsorptionNormSpectralModel
from gammapy.makers.utils import make_map_exposure_true_energy
from gammapy.maps import RegionNDMap, RegionGeom
from gammapy.maps import Map, WcsNDMap, MapAxis

In [ ]:
# Imports from this project
from gammapy.astro.darkmatter.utils import SigmaVEstimator, DMDatasetOnOff
from gammapy.astro.darkmatter import DarkMatterAnnihilationSpectralModel


## Define parameters for simulated observation

In [ ]:
livetime = 300 * u.h
offset = 0.5 * u.deg
FOVLON= 0 * u.deg
FOVLAT= 0 * u.deg

# Energy from 0.01 to 100 TeV with 20 bins/decade
energy = np.logspace(-1.8, 1.5, 20) * u.TeV

# Energy true wider range and higher number of bins
energy_true = np.logspace(-2, 2, 100) * u.TeV

# Factors for the DM model
JFAC = 1.42e19 * u.Unit("GeV2 cm-5") # Draco from Aguirre-Santaella+20
mDM = 1000*u.Unit("GeV")
channel = "b"
redshift = 0

## Build IRFs with the defined parameters

In [ ]:
# Load IRFs
filename = (
    "$GAMMAPY_DATA/cta-1dc/caldb/data/cta/1dc/bcf/South_z20_50h/irf_file.fits"
)
cta_irf = load_cta_irfs(filename)

In [ ]:
aeff = cta_irf["aeff"].to_effective_area_table(offset=offset, energy=energy)
aeff.plot()
plt.loglog()
print(cta_irf["aeff"].data)

## Build the background counts from PSF and IRF background

In [ ]:
containment = 0.68
energies = np.sqrt(energy[1:] * energy[:-1])
psf = cta_irf["psf"].to_energy_dependent_table_psf(theta=offset)
on_radii = psf.containment_radius(energy=energies, fraction=containment)
solid_angles = 2 * np.pi * (1 - np.cos(on_radii)) * u.sr

aeff.data.data *= containment

In [ ]:
bkg_data = cta_irf["bkg"].evaluate_integrate(
    fov_lon=FOVLON, fov_lat=FOVLAT, energy_reco=energy
)
axis = MapAxis.from_nodes(energies * u.TeV, interp="log", name="energy")
bkg = RegionNDMap.create(f"icrs;circle({FOVLON.value}, {FOVLAT.value}, {offset.value})", axes=[axis])
bkg.quantity = (bkg_data * solid_angles).to_value("h-1")*livetime.value

In [ ]:
bkg.plot()

## Simulate OnOff observation with DM emission model

In [ ]:
# DM Spatial Component
spatial_model = PointSpatialModel(
    lon_0="260.05 deg", lat_0="57.915 deg", frame="icrs"
)
spatial_model.lon_0.frozen=True
spatial_model.lat_0.frozen=True

In [ ]:
# DM Flux
spectral_model = DarkMatterAnnihilationSpectralModel(
    mass=mDM, 
    channel=channel, 
    jfactor=JFAC, 
    z=redshift
)

spectral_model = EBLAbsorptionNormSpectralModel.read_builtin(
    "dominguez",
    redshift=redshift
) * spectral_model

In [ ]:
spectral_model.parameters.to_table()

In [ ]:
sky_model = SkyModel(spatial_model=spatial_model, spectral_model=spectral_model)
models = Models([sky_model])

In [ ]:
# construct super simple exposure
geom = RegionGeom(region=spatial_model.to_region(), axes=[aeff.data.axes['energy_true']])
exposure = make_map_exposure_true_energy(spatial_model.position, livetime, cta_irf['aeff'], geom)

In [ ]:
dataset = DMDatasetOnOff(
    counts=bkg,
    counts_off=bkg,
    models=models,
    exposure=exposure,
    acceptance=1,
    acceptance_off=3
)

In [ ]:
nuisance = dict(
    j=JFAC,
    jobs=JFAC,
    sigmaj=0.3*JFAC,
    sigmatau=0.01,
    # width=5 # default value / optional param 
)
dataset.nuisance = nuisance

## The SigmaVEstimator

**Enable inspection**

In [ ]:
#import logging
#logging.basicConfig()
#logging.getLogger("gammapy.astro.darkmatter.utils").setLevel("WARNING")
#logging.getLogger("gammapy.astro.darkmatter.utils").setLevel("DEBUG")
import warnings

**Instatiate estimator**

In [ ]:
masses = [100, 200, 500, 1000, 5000, 10000, 50000]*u.GeV
channels = ["b", "tau", "Z"] 
estimator = SigmaVEstimator(dataset, masses, channels)

In [ ]:
estimator.dataset.models.parameters.to_table()

**Run estimator and fetch results**

In [ ]:
%%time
# stat_profile_opts=dict(bounds=(-25, 100), nvalues=50)     # default param
# if nuisance = True the process takes the nuisance parameters into account
warnings.filterwarnings("ignore")
result = estimator.run(runs=5, nuisance=False)

**Display results for channel b and run 1**

In [ ]:
cols = ["mass", "sigma_v", "sv_ul", "sv_best", "j_best"]
result["runs"]["b"][0][cols]

**Plot likelihood profile for a specific fit**

In [ ]:
idx = np.argwhere(masses.value==100)
profile = result["runs"]["b"][0]["statprofile"][idx][0][0]

In [ ]:
plt.plot(profile["sv_scan"], profile["stat_scan"]);
plt.xlabel('Statistic Value')

## Obtained results

In [ ]:
result["mean"]["b"]

In [ ]:
plt.figure(figsize=(9,7))
plt.ylim(1e-26, 1e-23)
plt.xlim(90, 60000)
plt.ylabel(r"<$\sigma_{ann}v$> (cm$^3$s$^{-1}$)", fontsize=12)
plt.xlabel("m$_{DM}$ (GeV)", fontsize=12)
plt.title(r'm$_{DM}$= 1 TeV, $b\bar{b}$')
plt.hlines(3e-26, 100, 50000, ls="--")


for ch in channels:
    plt.loglog(
        result["mean"][ch]["mass"].data,
        result["mean"][ch]["sigma_v"].data,
        ls=":",
        label='channel {}'.format(ch)
       )
plt.legend();